<a href="https://colab.research.google.com/github/Hyuncastl/MACHINE_LEARNING/blob/main/12_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import urllib
import zipfile

url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"

urllib.request.urlretrieve(url,'cats_and_dogs_filtered.zip')

local_zip = '/content/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()


train_dir = '/content/cats_and_dogs_filtered/train'
valid_dir = '/content/cats_and_dogs_filtered/validation'

train_datagen = ImageDataGenerator(rescale=1/255.,
                                   horizontal_flip=True,
                                   rotation_range=35,
                                   zoom_range=0.2)
train_gen = train_datagen.flow_from_directory(train_dir,
                                              batch_size=32,
                                              target_size=(224, 224),
                                              classes=['cats', 'dogs'],
                                              class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1/255.,
                                   horizontal_flip=True,
                                   rotation_range=35,
                                   zoom_range=0.2)
valid_gen = valid_datagen.flow_from_directory(valid_dir,
                                              batch_size=32,
                                              target_size=(224, 224),
                                              classes=['cats', 'dogs'],
                                              class_mode='binary')

def solution_model(): 
  model = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy']
                  )

  checkpoint_path = 'temp_checkpoint.ckpt'
  checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only=True,
                                 save_best_only=True,
                                 monitor='val_loss',
                                 verbose=1
                                 )
  model.fit(train_gen, validation_data=valid_gen,
              epochs=10, callbacks=[checkpoint])

  model.load_weights(checkpoint_path)

   
  return model


# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-cats-vs-dogs.h5")


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


41/63 [==================>...........] - ETA: 1:21 - loss: 31.6592 - accuracy: 0.5378